In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
import plotly.express as px
import os
import requests
from tqdm import tqdm

In [18]:
dir = "projects/repo/Layoff-Analysis/"

In [3]:
layoff_data_df = pd.read_csv(dir + "data/layoffs.fyi_full_data.csv")

In [4]:
post_ipo_df = layoff_data_df[layoff_data_df["Stage"] == "Post-IPO"]

In [ ]:
post_ipo_df.to_csv(dir + "/data2/post_ipo_data.csv", index=False)

In [8]:
post_ipo_df.info

<bound method DataFrame.info of              Company         Location HQ  # Laid Off       Date     %  \
0              Intel          Sacramento     22000.0  4/23/2025   20%   
3           AppLovin         SF Bay Area        97.0  4/16/2025   NaN   
6             Sonder         SF Bay Area         NaN  4/14/2025   NaN   
8             Google         SF Bay Area         NaN  4/11/2025   NaN   
9     Marin Software         SF Bay Area         NaN  4/10/2025  100%   
...              ...                 ...         ...        ...   ...   
3824          Redfin             Seattle       236.0   4/7/2020    7%   
3845        Bitfarms     Quebec,Non-U.S.         NaN   4/6/2020   NaN   
3875        MindBody     San Luis Obispo       700.0   4/2/2020   35%   
3944           Maven             Seattle        31.0  3/30/2020    9%   
3968            Mogo  Vancouver,Non-U.S.        78.0  3/27/2020   30%   

         Industry                                             Source  \
0        Hardware  

In [9]:
!pip install yahooquery

  Obtaining dependency information for yahooquery from https://files.pythonhosted.org/packages/d1/a9/9a06f31cc068c7997b63a358f94ed433afb28599ef63bdcd5333db4a19d8/yahooquery-2.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for requests-futures<2.0.0,>=1.0.1 from https://files.pythonhosted.org/packages/91/23/7c1096731c15c83826cb0dd42078b561a838aed44c36f370aeb815168106/requests_futures-1.0.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.2 MB/s eta 0:00:00


In [22]:
def get_ticker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={"User-Agent": user_agent})
    data = res.json()

    if data.get("quotes") and len(data["quotes"]) > 0:
        return data["quotes"][0]["symbol"]
    else:
        return "None"


In [26]:
tqdm.pandas()
post_ipo_df["Ticker"] = post_ipo_df["Company"].progress_apply(get_ticker)

100%|█████████████████████████████████████████| 898/898 [03:18<00:00,  4.53it/s]
/var/folders/w9/488y8jbj0plfk1q3x8hrsdgw0000gn/T/ipykernel_53193/2947765307.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_ipo_df["Ticker"] = post_ipo_df["Company"].progress_apply(get_ticker)


In [28]:
post_ipo_df = post_ipo_df[post_ipo_df["Ticker"].notnull()]

In [29]:
post_ipo_df

,Company,Location HQ,# Laid Off,Date,%,Industry,Source,Stage,$ Raised (mm),Country,Date Added,Ticker
0,Intel,Sacramento,22000.0,2025-04-23,20%,Hardware,https://www.bloomberg.com/news/articles/2025-0...,Post-IPO,$12,United States,4/23/2025,INTC
3,AppLovin,SF Bay Area,97.0,2025-04-16,NaN,Marketing,https://www.pocketgamer.biz/applovin-lays-off-...,Post-IPO,$1600,United States,4/18/2025,APP
6,Sonder,SF Bay Area,NaN,2025-04-14,NaN,Travel,https://skift.com/2025/04/14/sonder-announces-...,Post-IPO,$839,United States,4/16/2025,SOND
8,Google,SF Bay Area,NaN,2025-04-11,NaN,Consumer,https://www.reuters.com/technology/google-lays...,Post-IPO,$26,United States,4/16/2025,GOOG
9,Marin Software,SF Bay Area,NaN,2025-04-10,100%,Marketing,https://searchengineland.com/marin-software-sh...,Post-IPO,$99,United States,4/16/2025,MRIN
...,...,...,...,...,...,...,...,...,...,...,...,...
3824,Redfin,Seattle,236.0,2020-04-07,7%,Real Estate,https://www.geekwire.com/2020/redfin-lays-off-...,Post-IPO,$319,United States,4/7/2020,RDFN
3845,Bitfarms,"Quebec,Non-U.S.",NaN,2020-04-06,NaN,Crypto,https://coingeek.com/job-losses-loom-at-bitfar...,Post-IPO,$25,Canada,4/8/2020,BITF
3875,MindBody,San Luis Obispo,700.0,2020-04-02,35%,Fitness,https://www.sanluisobispo.com/news/business/ar...,Post-IPO,$114,United States,4/2/2020,None
3944,Maven,Seattle,31.0,2020-03-30,9%,Media,https://www.businesswire.com/news/home/2020033...,Post-IPO,$77,United States,4/7/2020,96W.F


In [30]:
post_ipo_df.to_csv(dir + "/data2/clean_with_ticker.csv", index=False)